In [1]:
import pandas as pd
print(f"Pandas: {pd.__version__}")
import numpy as np
print(f"Numpy: {np.__version__}")
import tensorflow as tf
print(f"Tensorflow: {tf.__version__}")
from tensorflow import keras
print(f"Keras: {keras.__version__}")
import sklearn
print(f"Sklearn: {sklearn.__version__}")
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)
import collections
import yaml
import re
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input
train = pd.read_pickle('objects/train.pkl')
print(f'Training data: {train.head()}')

Pandas: 1.4.1
Numpy: 1.22.3
Tensorflow: 2.8.0
Keras: 2.8.0
Sklearn: 1.0.2
Training data:                                            Utterance           Intent
0  [charge, iphone, still, power, percentage, dec...          battery
1                                 [forgot, my, pass]  forgot_password
2                            [payment, not, through]          payment
3                                     [want, update]           update
4                                [need, information]             info


In [2]:
train['Intent']=train['Intent'].astype('category')


In [3]:
train['Intent'].head(5)

0            battery
1    forgot_password
2            payment
3             update
4               info
Name: Intent, dtype: category
Categories (11, object): ['battery', 'challenge_robot', 'forgot_password', 'goodbye', ..., 'payment', 'repair', 'speak_representative', 'update']

In [4]:
one_hot_encoded_data = pd.get_dummies(train, columns = ['Intent'],sparse=True)
one_hot_encoded_data.replace({"NA": np.nan},inplace=True)
print(one_hot_encoded_data)

                                               Utterance  Intent_battery  \
0      [charge, iphone, still, power, percentage, dec...               1   
1                                     [forgot, my, pass]               0   
2                                [payment, not, through]               0   
3                                         [want, update]               0   
4                                    [need, information]               0   
...                                                  ...             ...   
10995  [get, info, work, keychain, access, high, sier...               0   
10996        [talk, human, please, person, someone real]               0   
10997                                              [hii]               0   
10998                                           [thanks]               0   
10999                             [I, do not, like, you]               0   

       Intent_challenge_robot  Intent_forgot_password  Intent_goodbye  \
0             

C:\Users\muthu\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\sparse\array.py:1723: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result[self.sp_index.indices] = op(self.sp_values, other)


In [5]:
intent_class=[]
for i in one_hot_encoded_data:
    intent_class.append(i)
print(intent_class)

['Utterance', 'Intent_battery', 'Intent_challenge_robot', 'Intent_forgot_password', 'Intent_goodbye', 'Intent_greeting', 'Intent_info', 'Intent_lost_replace', 'Intent_payment', 'Intent_repair', 'Intent_speak_representative', 'Intent_update']


In [6]:
one_hot_encoded_data.head(5)

,Utterance,Intent_battery,Intent_challenge_robot,Intent_forgot_password,Intent_goodbye,Intent_greeting,Intent_info,Intent_lost_replace,Intent_payment,Intent_repair,Intent_speak_representative,Intent_update
0,"[charge, iphone, still, power, percentage, dec...",1,0,0,0,0,0,0,0,0,0,0
1,"[forgot, my, pass]",0,0,1,0,0,0,0,0,0,0,0
2,"[payment, not, through]",0,0,0,0,0,0,0,1,0,0,0
3,"[want, update]",0,0,0,0,0,0,0,0,0,0,1
4,"[need, information]",0,0,0,0,0,1,0,0,0,0,0


In [7]:

one_hot_encoded_data=one_hot_encoded_data.drop("Utterance",axis=1)

labels=one_hot_encoded_data
labels


,Intent_battery,Intent_challenge_robot,Intent_forgot_password,Intent_goodbye,Intent_greeting,Intent_info,Intent_lost_replace,Intent_payment,Intent_repair,Intent_speak_representative,Intent_update
0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
10995,0,0,0,0,0,0,1,0,0,0,0
10996,0,0,0,0,0,0,0,0,0,1,0
10997,0,0,0,0,1,0,0,0,0,0,0
10998,0,0,0,1,0,0,0,0,0,0,0


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


from keras.preprocessing.text import hashing_trick
from keras.preprocessing.text import text_to_word_sequence


X_train, X_val, y_train, y_val = train_test_split(train['Utterance'], labels, test_size = 0.3, 
                                                   shuffle = True, random_state = 7)
print(f'\nShape checks:\nX_train: {X_train.shape} X_val: {X_val.shape}\ny_train: {y_train.shape} y_val: {y_val.shape}')


Shape checks:
X_train: (7700,) X_val: (3300,)
y_train: (7700, 11) y_val: (3300, 11)


In [9]:

y_train.head(5)

,Intent_battery,Intent_challenge_robot,Intent_forgot_password,Intent_goodbye,Intent_greeting,Intent_info,Intent_lost_replace,Intent_payment,Intent_repair,Intent_speak_representative,Intent_update
6177,0,0,0,0,0,0,1,0,0,0,0
5635,0,0,0,0,0,0,0,0,0,0,1
9981,0,0,0,0,0,1,0,0,0,0,0
10098,1,0,0,0,0,0,0,0,0,0,0
8290,0,0,0,0,0,0,0,0,0,1,0


In [10]:
X_train.head(5)

6177     [back, cover, come, series, apple, watch, unde...
5635     [update, severely, ruin, phone, help, fix, angry]
9981     [siri, utterly, useless, never, get, anything,...
10098            [already, do, dm, response, tht, side, ️]
8290           [talk, human, please, person, someone real]
Name: Utterance, dtype: object

In [11]:
y_train
y_val

,Intent_battery,Intent_challenge_robot,Intent_forgot_password,Intent_goodbye,Intent_greeting,Intent_info,Intent_lost_replace,Intent_payment,Intent_repair,Intent_speak_representative,Intent_update
6242,0,0,0,0,0,0,0,0,1,0,0
158,0,0,0,0,0,1,0,0,0,0,0
8518,0,0,0,0,0,1,0,0,0,0,0
10845,0,1,0,0,0,0,0,0,0,0,0
4369,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
9597,0,0,0,0,0,0,0,0,1,0,0
9376,0,0,0,0,0,1,0,0,0,0,0
477,0,0,0,0,0,1,0,0,0,0,0
9785,0,0,0,0,0,0,1,0,0,0,0


In [12]:
t = Tokenizer()
t.fit_on_texts(X_train)

print("Document Count: \n{}\n".format(t.document_count))


def convert_to_padded(tokenizer, docs):
    ''' Taking in Keras API Tokenizer and documents and returns their padded version '''

    embedded = t.texts_to_sequences(docs)

    padded = pad_sequences(embedded, maxlen = max_length, padding = 'post')
    return padded


vocab_size = len(t.word_counts) + 1
print(f'Vocab size:\n{vocab_size}')


max_length = len(max(X_train, key = len))

print(f'Max length:\n{max_length}')

padded_X_train = convert_to_padded(tokenizer = t, docs = X_train)
padded_X_val = convert_to_padded(tokenizer = t, docs = X_val)

print(f'padded_X_train\n{padded_X_train}')
print(f'padded_X_val\n{padded_X_val}')

Document Count: 
7700

Vocab size:
4588
Max length:
32
padded_X_train
[[  68  220  183 ...    0    0    0]
 [   3  893  381 ...    0    0    0]
 [ 312 1207  486 ...    0    0    0]
 ...
 [ 283  178  621 ...    0    0    0]
 [  91    0    0 ...    0    0    0]
 [  12   17   10 ...    0    0    0]]
padded_X_val
[[ 10   7 141 ...   0   0   0]
 [ 49  21  45 ...   0   0   0]
 [ 49  21  45 ...   0   0   0]
 ...
 [ 89  60  78 ...   0   0   0]
 [258  14 331 ...   0   0   0]
 [ 46  47  96 ...   0   0   0]]


In [13]:
padded_X_train.shape, padded_X_val.shape, y_train.shape, y_val.shape

((7700, 32), (3300, 32), (7700, 11), (3300, 11))

In [14]:
padded_X_train[1]

array([  3, 893, 381,   4,  23,  14, 730,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0])

In [15]:
embeddings_index = {}
f = open('models/glove.twitter.27B/glove.twitter.27B.50d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193515 word vectors.


In [16]:

word_index = t.word_index
EMBEDDING_DIM = 50 


embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        
        embedding_matrix[i] = embedding_vector

In [17]:
embedding_matrix, embedding_matrix.shape

(array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.51418   , -0.19196001, -0.79962999, ..., -0.34937999,
         -1.08060002,  0.60174   ],
        [ 0.36294001, -0.26519001, -1.30439997, ..., -0.22002   ,
         -0.74168998,  0.56022   ],
        ...,
        [ 0.13165   ,  0.033388  ,  0.69450998, ...,  0.24797   ,
         -0.086261  ,  0.10927   ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.25253001,  0.60267001, -0.35211   , ...,  0.13484   ,
          0.37526   , -0.17753001]]),
 (4588, 50))

In [21]:
for i in y_val:
    t=i
    break
type(t)

str

In [18]:
y_val = y_val.astype('float32')
y_train = y_train.astype('float32')


In [ ]:
import tensorflow as tf


class ryz(tf.keras.losses.Loss):
    def __init__(self, batch_size: int = 32):
        super(ryz, self).__init__()
        self.batch_size = batch_size

    def call(self, y_true, y_pred):
        tmp = []
        for item in range(self.batch_size):
            tf.print(f'Working on batch: {item}\n')
            s = y_true[item, :]
            t = y_pred[item, :]
            n, m = len(s), len(t)
            dtw_matrix = []
            for i in range(n + 1):
                line = []
                for j in range(m + 1):
                    if i == 0 and j == 0:
                        line.append(0)
                    else:
                        line.append(np.inf)
                dtw_matrix.append(line)

            for i in range(1, n + 1):
                for j in range(1, m + 1):
                    cost = tf.abs(s[i - 1] - t[j - 1])
                    last_min = tf.reduce_min([dtw_matrix[i - 1][j], dtw_matrix[i][j - 1], dtw_matrix[i - 1][j - 1]])
                    dtw_matrix[i][j] = tf.cast(cost, dtype=tf.float32) + tf.cast(last_min, dtype=tf.float32)

            temp = []
            for i in range(len(dtw_matrix)):
                temp.append(tf.stack(dtw_matrix[i]))

            tmp.append(tf.stack(temp)[n, m])
        
        cce = tf.keras.losses.CategoricalCrossentropy()
        error=cce(y_true, y_pred)
        check=tf.math.greater(error,tmp)
        if check==True:
          return tf.reduce_mean(tmp)
        else:
          return tf.reduce_mean(error)

In [ ]:
import keras.backend as K
def get_f1(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val


y_val = y_val.astype('float32')
y_train = y_train.astype('float32')

In [ ]:
def make_model(vocab_size, max_token_length):
    ''' In this function I define all the layers of my neural network'''
    
    model = Sequential()
    #model.add(Input(shape = (32,), dtype = 'int32'))

    
    model.add(Embedding(vocab_size, embedding_matrix.shape[1], input_length = 32, 
                        trainable = False, weights = [embedding_matrix]))
    
    model.add(Bidirectional(LSTM(128,return_sequences= True)))
    model.add(LSTM(128)) 
    
    model.add(Dense(600, activation = "tanh",kernel_regularizer ='l2')) 
    
    
    # model.add(Dense(600, activation = "relu",kernel_regularizer ='l2'))
    model.add(Dense(300, activation = "tanh",kernel_regularizer ='l2'))
    model.add(Dense(150, activation = "tanh",kernel_regularizer ='l2'))
    # model.add(Dense(50, activation = "tanh",kernel_regularizer ='l2'))
    # model.add(Dense(64, activation = "relu",kernel_regularizer ='l2'))
    
    model.add(Dropout(0.5))
    

    model.add(Dense(11, activation = "sigmoid"))
    
    return model

In [ ]:
model = make_model(vocab_size, 32)
model.compile(loss = ryz(batch_size=1), 
              optimizer = "rmsprop", metrics = ["accuracy"])
model.summary()


filename = 'models/ryz_1.h5'


def scheduler(epoch, lr):
    if epoch < 12:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_sched_checkpoint = tf.keras.callbacks.LearningRateScheduler(scheduler)


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto',
    baseline=None, restore_best_weights=True
)



checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min')


hist = model.fit(padded_X_train, y_train, epochs = 20, batch_size = 32, 
                 validation_data = (padded_X_val, y_val), 
                 callbacks = [checkpoint, lr_sched_checkpoint, early_stopping])


In [ ]:
plt.figure(figsize=(10,7))
plt.plot(hist.history['val_loss'], label = 'Validation Loss', color = 'cyan')
plt.plot(hist.history['loss'], label = 'Training Loss', color = 'purple')
plt.title('Training Loss vs Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


plt.figure(figsize=(10,7))
plt.plot(hist.history['val_accuracy'], label = 'Validation Accuracy', color = 'cyan')
plt.plot(hist.history['accuracy'], label = 'Training Accuracy', color = 'purple')
plt.title('Training Accuracy vs Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [19]:

from keras.models import load_model
model = load_model('models/ryz_1.h5',compile=False)

In [45]:

t = Tokenizer()
t.fit_on_texts(X_train)

print("Document Count: \n{}\n".format(t.document_count))

def convert_to_padded(tokenizer, docs):
    ''' Taking in Keras API Tokenizer and documents and returns their padded version '''

    embedded = t.texts_to_sequences(docs)

    padded = pad_sequences(embedded, maxlen = max_length, padding = 'post')
    return padded


vocab_size = len(t.word_counts) + 1
print(f'Vocab size:\n{vocab_size}')


max_length = len(max(X_train, key = len))

print(f'Max length:\n{max_length}')

padded_X_train = convert_to_padded(tokenizer = t, docs = X_train)
padded_X_val = convert_to_padded(tokenizer = t, docs = X_val)

print(f'padded_X_train\n{padded_X_train}')
print(f'padded_X_val\n{padded_X_val}')

Document Count: 
7700

Vocab size:
4692
Max length:
32
padded_X_train
[[  16    8   40 ...    0    0    0]
 [  33  117    5 ...    0    0    0]
 [   6 1030 1237 ...    0    0    0]
 ...
 [  48 4691    7 ...    0    0    0]
 [ 202  181  478 ...    0    0    0]
 [  63   65  210 ...    0    0    0]]
padded_X_val
[[  8  52  19 ...   0   0   0]
 [271  16  25 ...   0   0   0]
 [108 114   0 ...   0   0   0]
 ...
 [ 16   8  40 ...   0   0   0]
 [206   5   3 ...   0   0   0]
 [  2   4   6 ...   0   0   0]]


In [65]:
intent_class=intent_class[1:]

In [67]:
def infer_intent(user_input):
    ''' Making a function that recieves a user input and outputs a 
    dictionary of predictions '''
    assert isinstance(user_input, str), 'User input must be a string!'
    user_input = [user_input]
    print(user_input)
    

    padded_text = convert_to_padded(t, user_input)
    x = padded_text[0]
    

    probs = model.predict(padded_text)

    
    

    classes = intent_class
    

    predictions = dict(zip(classes, probs[0]))
    sorted_predictions = {k: v for k, v in sorted(predictions.items(), key=lambda item: item[1], reverse = True)}
    
    return sorted_predictions

In [66]:
intent_class

['Intent_battery',
 'Intent_challenge_robot',
 'Intent_forgot_password',
 'Intent_goodbye',
 'Intent_greeting',
 'Intent_info',
 'Intent_lost_replace',
 'Intent_payment',
 'Intent_repair',
 'Intent_speak_representative',
 'Intent_update']

In [68]:
res=infer_intent("I want to know about the latest iphone")
print(res)

['I want to know about the latest iphone']
{'Intent_speak_representative': 0.99999166, 'Intent_challenge_robot': 0.9971344, 'Intent_info': 0.9896754, 'Intent_repair': 0.9893247, 'Intent_lost_replace': 0.9827001, 'Intent_goodbye': 0.920865, 'Intent_payment': 0.095944405, 'Intent_forgot_password': 0.054655164, 'Intent_battery': 0.03375557, 'Intent_greeting': 0.024140656, 'Intent_update': 0.0004130602}


In [69]:
X_test=[]
for i in X_val:
    temp=" ".join(i)
    X_test.append(temp)
# count=0  
# for i in X_test:
#     count+=1
#     if count==5:
#         break
#     else:
#         print(i)

In [70]:
# y_pred=infer_intent(X_val)
y_pred=map(infer_intent,X_test)

In [71]:
len(X_test)

3300

In [72]:
pred=list(y_pred)
for i in pred:
    print(i)
    break

['I do not like you']
['seriously can not hookup hdmi monitor mac pro seriously frak wrong']
['whats up']
['are you robot']
['please repair control center bluetooth completely turn']
['byeee']
['byeee']
['need information']
['update not working']
['can i speak agent person']
['what are macbook stats']
['try reset couple time work start get service back time']
['charger two hour almost wtf use iphone apple charger']
['try install new ossierra update macbook pro fail lock computer error message can not get back']
['okay also try recover apple id week progress idea disabled']
['talk human please person someone real']
['talk human please person someone real']
['good night']
['can i speak agent person']
['thank u cause iphone plus big piece glitchy shit drain battery']
['billing address somehow default shipping address place order change get cancel']
['can i speak agent person']
['are you robot']
['update phone keep crash fix use phone']
['let me talk to apple support']
['thank goodness set

In [73]:
for i in pred:
    t=i
    break

In [74]:
def dictchange(qwq):
    first_value = list(qwq.keys())[0]
    return(first_value)

In [75]:
y_pred=list(map(dictchange,pred))

In [76]:
X_train, X_val, y_train, y_val = train_test_split(train['Utterance'], train['Intent'], test_size = 0.3, 
                                                   shuffle = True, stratify = train['Intent'], random_state = 7)
print(f'\nShape checks:\nX_train: {X_train.shape} X_val: {X_val.shape}\ny_train: {y_train.shape} y_val: {y_val.shape}')


Shape checks:
X_train: (7700,) X_val: (3300,)
y_train: (7700,) y_val: (3300,)


In [77]:
y_val

2155         challenge_robot
7937            lost_replace
6014                greeting
5961         challenge_robot
385                  battery
                ...         
9245                  repair
5240                    info
3692    speak_representative
3776                  update
7871            lost_replace
Name: Intent, Length: 3300, dtype: category
Categories (11, object): ['battery', 'challenge_robot', 'forgot_password', 'goodbye', ..., 'payment', 'repair', 'speak_representative', 'update']

In [78]:
y_true=[]
for i in y_val:
    y_true.append(i)
    

In [79]:
y_pred
for i in y_true:
    print(i)
    break

challenge_robot


In [80]:
y_true

['challenge_robot',
 'lost_replace',
 'greeting',
 'challenge_robot',
 'battery',
 'goodbye',
 'goodbye',
 'info',
 'update',
 'speak_representative',
 'info',
 'battery',
 'battery',
 'forgot_password',
 'info',
 'speak_representative',
 'speak_representative',
 'greeting',
 'speak_representative',
 'battery',
 'payment',
 'speak_representative',
 'challenge_robot',
 'update',
 'speak_representative',
 'lost_replace',
 'goodbye',
 'goodbye',
 'lost_replace',
 'speak_representative',
 'goodbye',
 'info',
 'battery',
 'challenge_robot',
 'payment',
 'battery',
 'battery',
 'greeting',
 'goodbye',
 'repair',
 'forgot_password',
 'payment',
 'forgot_password',
 'greeting',
 'forgot_password',
 'challenge_robot',
 'speak_representative',
 'repair',
 'speak_representative',
 'repair',
 'speak_representative',
 'challenge_robot',
 'update',
 'speak_representative',
 'payment',
 'update',
 'speak_representative',
 'forgot_password',
 'payment',
 'speak_representative',
 'speak_representative'

In [81]:
testing=pd.DataFrame()
testing['1']=y_pred
testing['2']=y_true
testing.head(5)

,1,2
0,Intent_challenge_robot,challenge_robot
1,Intent_battery,lost_replace
2,Intent_greeting,greeting
3,Intent_challenge_robot,challenge_robot
4,Intent_battery,battery


In [82]:
y_pred

['Intent_challenge_robot',
 'Intent_battery',
 'Intent_greeting',
 'Intent_challenge_robot',
 'Intent_battery',
 'Intent_goodbye',
 'Intent_goodbye',
 'Intent_info',
 'Intent_update',
 'Intent_speak_representative',
 'Intent_info',
 'Intent_forgot_password',
 'Intent_battery',
 'Intent_forgot_password',
 'Intent_update',
 'Intent_speak_representative',
 'Intent_speak_representative',
 'Intent_greeting',
 'Intent_speak_representative',
 'Intent_battery',
 'Intent_payment',
 'Intent_speak_representative',
 'Intent_challenge_robot',
 'Intent_update',
 'Intent_speak_representative',
 'Intent_lost_replace',
 'Intent_goodbye',
 'Intent_goodbye',
 'Intent_forgot_password',
 'Intent_speak_representative',
 'Intent_goodbye',
 'Intent_repair',
 'Intent_forgot_password',
 'Intent_challenge_robot',
 'Intent_payment',
 'Intent_battery',
 'Intent_update',
 'Intent_greeting',
 'Intent_goodbye',
 'Intent_repair',
 'Intent_forgot_password',
 'Intent_payment',
 'Intent_forgot_password',
 'Intent_greetin

In [83]:
predictions=[]
for i in y_pred:
    temp=i[7:]
    predictions.append(temp)

In [84]:
y_pred=predictions


In [85]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score

precision = precision_score(y_true, y_pred, average='weighted', labels=np.unique(y_pred))
recall = recall_score(y_true, y_pred, average='macro',labels=np.unique(y_pred))
score = f1_score(y_true, y_pred, average='micro')
accuracy=accuracy_score(y_true, y_pred,normalize=True)
print('Precision: %.3f' % precision)
print('Recall: %.3f' % recall)
print('F-Measure: %.3f' % score)
print('Accuracy: %.3f' % accuracy)

Precision: 0.815
Recall: 0.804
F-Measure: 0.804
Accuracy: 0.804
